In [21]:
import os
import pandas as pd
import openpyxl
import re

def read_excel_sheets(file_path):
    """Read sheet names from an Excel file."""
    try:
        workbook = openpyxl.load_workbook(file_path)
        return workbook.sheetnames
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return []

def extract_data_from_sheet(file_path, sheet_name):
    """Extract specific data from an Excel sheet."""
    try:
        workbook = openpyxl.load_workbook(file_path)
        sheet = workbook[sheet_name]
        data = {
            'file_name': os.path.basename(file_path),
            'Process_Name': sheet_name,
            'P/N': [],
            '工序名稱': [],
            'H_row': None,  # 存储 (H) 的行号
            'M_row': None,  # 存储 (M) 的行号
            'L_row': None,  # 存储 (L) 的行号
            'End_row': None,  # End row num
            'non_empty_count_H_to_M': 0,  # 统计 (H) 到 (M) 之间的非空单元格数量
            'non_empty_count_M_to_L': 0,  # 统计 (M) 到 (L) 之间的非空单元格数量
            'non_empty_count_L_to_end': 0  # 统计 (L) 到 结束行 之间的非空单元格数量
        }
        
        # 遍历 A 列，找到 (H), (M), (L) 的行号
        for row in sheet.iter_rows(min_row=1, max_col=1, max_row=sheet.max_row):
            cell = row[0]  # A 列的单元格
            cell_value = str(cell.value) if cell.value is not None else ""
            if '(H)' in cell_value:
                data['H_row'] = cell.row
            if '(M)' in cell_value:
                data['M_row'] = cell.row
            if '(L)' in cell_value:
                data['L_row'] = cell.row
            if 'Form No' in cell_value:
                data['End_row'] = cell.row
        
        # 如果找到了 (H), (M), (L) 的行号
        if data['H_row'] and data['M_row'] and data['L_row']:
            # 统计 (H) 到 (M) 之间的非空单元格数量
            data['non_empty_count_H_to_M'] = count_numeric_cells(sheet, data['H_row'], data['M_row'], f"{data['file_name']} - {data['sheet_name']} (H to M)")
            # 统计 (M) 到 (L) 之间的非空单元格数量
            data['non_empty_count_M_to_L'] = count_numeric_cells(sheet, data['M_row'], data['L_row'], f"{data['file_name']} - {data['sheet_name']} (M to L)")
            # 统计 (L) 到 结束行 之间的非空单元格数量
            data['non_empty_count_L_to_end'] = count_numeric_cells(sheet, data['L_row'], data['End_row'], f"{data['file_name']} - {data['sheet_name']} (L to End)")
        
        return data
    except Exception as e:
        print(f"Error extracting data from {file_path} - {sheet_name}: {e}")
        return {}

def count_numeric_cells(sheet, start_row, end_row, section_name):
    """Count cells that contain numeric characters followed by '*' or a single or two numeric characters between two rows in column A."""
    count = 0
    for row in sheet.iter_rows(min_row=start_row + 1, max_row=end_row - 1, min_col=1, max_col=1):
        cell = row[0]  # 获取A列的单元格
        cell_value = str(cell.value) if cell.value is not None else ""
        if re.search(r'\d{1,2}\*$', cell_value) or re.fullmatch(r'\d{1,2}', cell_value):
            count += 1
    return count

def main(folder_path, output_file):
    all_data = []
    file_count = 0
    total_files = sum(1 for file in os.listdir(folder_path) if file.endswith('.xlsx'))
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith('.xlsx'):
            file_count += 1
            print(f"Processing file {file_count}/{total_files}: {file_name}")
            sheet_names = read_excel_sheets(file_path)
            for sheet_name in sheet_names:
                data = extract_data_from_sheet(file_path, sheet_name)
                all_data.append(data)
    
    df = pd.DataFrame(all_data)
    print("\nSummary DataFrame:")
    print(df)
    
    # Write the DataFrame to a new Excel file
    df.to_excel(output_file, index=False)
    print(f"Data has been written to {output_file}")

# Update the folder path to the specific folder containing Excel files
# Specify the output file path
main('S:\\Santo\\QA\\3_Plan_WI\\3-IPQC报表\\PCP版IPQC报表', 'C:\\Jim.Huang\\IPQC\\Test\\output_summary.xlsx')

Processing file 1/1270: 301743003 01PCP&IPQC-Kavin.xlsx
Processing file 2/1270: 301743004 03-PCP-Lv.xlsx
Processing file 3/1270: 301743006 03-PCP&IPQC-Danny.xlsx
Processing file 4/1270: 302565001 03-PCP&IPQC-Danny.xlsx
Processing file 5/1270: 302568003 02 PCP&IPQC .xlsx
Processing file 6/1270: 305180002 04 PCP-Sunny Sun.xlsx
Processing file 7/1270: 307148001 03 PCP&IPQC.xlsx
Processing file 8/1270: 307821001 02 -PCP&IPQC-Nessen.xlsx
Processing file 9/1270: 313152004 02-PCP版IPQC--Davin.xlsx
Processing file 10/1270: 314110001 04-PCP&IPQC-GB.Li.xlsx
Processing file 11/1270: 314111001 04-PCP&IPQC-LF.xlsx
Processing file 12/1270: 316428003 01 PCP-MP.xlsx
Processing file 13/1270: 316438003 01 PCP MP.xlsx
Processing file 14/1270: 316438003 01-PCP&IPQC-Nessen.xlsx
Processing file 15/1270: 318689002 03-PCP-20241230-Jh.huang.xlsx
Processing file 16/1270: 600990001 01PCP&IPQC.xlsx
Processing file 17/1270: 600999007 01-PCP&IPQC-LF.xlsx
Processing file 18/1270: 610027003 03-PCP-Lv.xlsx
Processing f

C:\Users\Jim.huang\AppData\Local\anaconda_new\Lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Unknown extension is not supported and will be removed
  ws_parser.bind_all()
C:\Users\Jim.huang\AppData\Local\anaconda_new\Lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Conditional Formatting extension is not supported and will be removed
  ws_parser.bind_all()


Processing file 399/1270: 621282010 01PCP&IPQC-SR.Yan.xlsx
Processing file 400/1270: 621282011 01 PCP-Sunny.Sun.xlsx
Processing file 401/1270: 621323002 01PCP&IPQC-Dang.xlsx
Processing file 402/1270: 621326011 02-PCP&IPQC-Danny.xlsx
Processing file 403/1270: 621331001 01 PCP&IPQC .xlsx
Processing file 404/1270: 621349005 04-PCP-Lv.xlsx
Processing file 405/1270: 621361001-02-PCP&Martin.xlsx
Processing file 406/1270: 621361002 03-PCP&IPQC-GB.Li.xlsx
Processing file 407/1270: 621387001 01-PCP&IPQC-Nessen.xlsx
Processing file 408/1270: 621390001 03 PCP-Sunny Sun.xlsx
Processing file 409/1270: 621391001-01-PCP&Martin.xlsx
Processing file 410/1270: 621392001 01-PCP&IPQC-GB.Li.xlsx
Processing file 411/1270: 621399002 03-PCP&IPQC-LF.xlsx
Processing file 412/1270: 621400001 01-PCP&IPQC-LF.xlsx
Processing file 413/1270: 621418001 02PCP&IPQC-Kavin.xlsx
Processing file 414/1270: 621451001 01 PCP&IPQC.xlsx
Processing file 415/1270: 621467001 01-PCP-Lv.xlsx
Processing file 416/1270: 621473003 01-PCP

C:\Users\Jim.huang\AppData\Local\anaconda_new\Lib\site-packages\openpyxl\reader\drawings.py:63: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)


Processing file 652/1270: 641673003-01-PCP&Martin.xlsx
Processing file 653/1270: 641722001 01 PCP&IPQC .xlsx
Processing file 654/1270: 641724001 02-PCP&IPQC-Nessen.xlsx
Processing file 655/1270: 641750002 04 PCP-Sunny Sun.xlsx
Processing file 656/1270: 641795002 03-PCP&IPQC-LF.xlsx
Processing file 657/1270: 641816001 01-PCP-Lv.xlsx
Processing file 658/1270: 641817001 01-PCP-Jh.huang.xlsx
Processing file 659/1270: 641824001 01-PCP-Kevin.Xiao.xlsx
Processing file 660/1270: 641825001 01-PCP&IPQC-YL.He.xlsx
Processing file 661/1270: 641831003 02 PCP.xlsx
Processing file 662/1270: 641841005 01-PCP&IPQC-LF.xlsx
Processing file 663/1270: 641842003-01-PCP&刘雄.xlsx
Processing file 664/1270: 641871001 09-PCP-Sunny.Sun.xlsx
Processing file 665/1270: 641871002 01PCP&IPQC-Kavin.xlsx
Processing file 666/1270: 641872001 07-PCP-Sunny.Sun.xlsx
Processing file 667/1270: 641899002 02 PCP&IPQC .xlsx
Processing file 668/1270: 641981001-01-PCP&Martin.xlsx
Processing file 669/1270: 641982001 01-PCP&IPQC-GB.Li